#1 导入NeMo NLP工具类及相关工具库

In [1]:
!pip install Cython
!pip install nemo_toolkit['all']

     |████████████████████████████████| 1.2 MB 7.1 MB/s 
     |████████████████████████████████| 145 kB 59.4 MB/s 
     |████████████████████████████████| 2.5 MB 29.8 MB/s 
     |████████████████████████████████| 108 kB 74.7 MB/s 
     |████████████████████████████████| 12.3 MB 178 kB/s 
     |████████████████████████████████| 765 kB 59.1 MB/s 
     |████████████████████████████████| 1.2 MB 62.3 MB/s 
     |████████████████████████████████| 813 kB 64.6 MB/s 
     |████████████████████████████████| 74 kB 4.8 MB/s 
     |████████████████████████████████| 272 kB 83.7 MB/s 
     |████████████████████████████████| 895 kB 41.3 MB/s 
     |████████████████████████████████| 3.1 MB 76.0 MB/s 
     |████████████████████████████████| 97 kB 10.3 MB/s 
     |████████████████████████████████| 47 kB 6.0 MB/s 
     |████████████████████████████████| 541 kB 68.9 MB/s 
     |████████████████████████████████| 1.4 MB 58.2 MB/s 
     |████████████████████████████████| 1.5 MB 20.2 MB/s 
     |██████████████

In [2]:
from nemo.collections import nlp as nemo_nlp
from nemo.utils.exp_manager import exp_manager #模型训练管理工具

import os
import wget 
import torch
import pytorch_lightning as pl
from omegaconf import OmegaConf #配置文件工具库

[NeMo W 2021-08-22 14:02:52 optimizers:47] Apex was not found. Using the lamb optimizer will error out.
[NeMo W 2021-08-22 14:02:56 experimental:28] Module <class 'nemo.collections.nlp.data.text_normalization.decoder_dataset.TextNormalizationDecoderDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2021-08-22 14:02:56 experimental:28] Module <class 'nemo.collections.nlp.data.text_normalization.tagger_dataset.TextNormalizationTaggerDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2021-08-22 14:02:56 experimental:28] Module <class 'nemo.collections.nlp.data.text_normalization.test_dataset.TextNormalizationTestDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


[NeMo W 2021-08-22 14:02:57 experimental:28] Module <class 'nemo.collections.nlp.models.duplex_text_normalization.duplex_decoder.DuplexDecoderModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2021-08-22 14:02:57 experimental:28] Module <class 'nemo.collections.nlp.models.duplex_text_normalization.duplex_tagger.DuplexTaggerModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[NeMo W 2021-08-22 14:02:57 experimental:28] Module <class 'nemo.collections.nlp.models.duplex_text_normalization.duplex_tn.DuplexTextNormalizationModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.


In [6]:
!pwd
!wget -c https://dl.fbaipublicfiles.com/glue/data/SST-2.zip

/content
--2021-08-22 14:06:06--  https://dl.fbaipublicfiles.com/glue/data/SST-2.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.74.142, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7439277 (7.1M) [application/zip]
Saving to: ‘SST-2.zip’

SST-2.zip           100%[===================>]   7.09M  28.8MB/s    in 0.2s    

2021-08-22 14:06:07 (28.8 MB/s) - ‘SST-2.zip’ saved [7439277/7439277]



#2 创建用来存放数据集和配置文件的目录

In [8]:
#调用系统模块定义并创建用来存放数据集和配置文件的目录
DATA_DIR = "/content"
WORK_DIR = "/content"
os.environ['DATA_DIR'] = DATA_DIR
#! unzip -o SST-2.zip -d {DATA_DIR}
os.makedirs(WORK_DIR, exist_ok=True)
os.makedirs(DATA_DIR, exist_ok=True)

#打印数据集中的前n行进行简单查看
print('Contents (first 5 lines) of train.tsv:')
! head -n 20 {DATA_DIR}/SST-2/train.tsv

print('\nContents (first 5 lines) of test.tsv:')
! head -n 10 {DATA_DIR}/SST-2/test.tsv

Contents (first 5 lines) of train.tsv:
sentence	label
hide new secretions from the parental units 	0
contains no wit , only labored gags 	0
that loves its characters and communicates something rather beautiful about human nature 	1
remains utterly satisfied to remain the same throughout 	0
on the worst revenge-of-the-nerds clichés the filmmakers could dredge up 	0
that 's far too tragic to merit such superficial treatment 	0
demonstrates that the director of such hollywood blockbusters as patriot games can still turn out a small , personal film with an emotional wallop . 	1
of saucy 	1
a depressed fifteen-year-old 's suicidal poetry 	0
are more deeply thought through than in most ` right-thinking ' films 	1
goes to absurd lengths 	0
for those moviegoers who complain that ` they do n't make movies like they used to anymore 	0
the part where nothing 's happening , 	0
saw how bad this movie was 	0
lend some dignity to a dumb story 	0
the greatest musicians 	1
cold movie 	0
with his usual 

#3 加载模型的配置文件并打印

In [9]:
#定义模型配置文件的路径
MODEL_CONFIG = "text_classification_config.yaml"
CONFIG_DIR = WORK_DIR + '/configs/'
os.makedirs(CONFIG_DIR, exist_ok=True)
#判断配置文件是否已经存在
if not os.path.exists(CONFIG_DIR + MODEL_CONFIG):
    print('Downloading config file...')
    wget.download('https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/nlp/text_classification/conf/' + MODEL_CONFIG, CONFIG_DIR)
    print('Config file downloaded!')
else:
    print ('config file already exists')
config_path = f'{WORK_DIR}/configs/{MODEL_CONFIG}'
print(config_path)
# 通过OmegaConf工具库加载模型配置文件，定义config对象
config = OmegaConf.load(config_path)
print(OmegaConf.to_yaml(config))

Config file downloaded!
/content/configs/text_classification_config.yaml
trainer:
  gpus: 1
  num_nodes: 1
  max_epochs: 100
  max_steps: null
  accumulate_grad_batches: 1
  gradient_clip_val: 0.0
  amp_level: O0
  precision: 32
  accelerator: ddp
  log_every_n_steps: 1
  val_check_interval: 1.0
  resume_from_checkpoint: null
  num_sanity_val_steps: 0
  checkpoint_callback: false
  logger: false
model:
  nemo_path: text_classification_model.nemo
  tokenizer:
    tokenizer_name: ${model.language_model.pretrained_model_name}
    vocab_file: null
    tokenizer_model: null
    special_tokens: null
  language_model:
    pretrained_model_name: bert-base-uncased
    lm_checkpoint: null
    config_file: null
    config: null
  classifier_head:
    num_output_layers: 2
    fc_dropout: 0.1
  class_labels:
    class_labels_file: null
  dataset:
    num_classes: ???
    do_lower_case: false
    max_seq_length: 256
    class_balancing: null
    use_cache: false
  train_ds:
    file_path: null
    b

#4 设置配置文件参数，指定数据集

In [11]:
config.model.dataset.num_classes=2 #通过config来定义数据集分类的类别：2分类任务
config.model.train_ds.file_path = os.path.join(DATA_DIR, 'SST-2/train.tsv')#指定训练集路径
config.model.validation_ds.file_path = os.path.join(DATA_DIR, 'SST-2/dev.tsv')#指定验证集路径
print("Train dataloader's config: \n")
print(OmegaConf.to_yaml(config.model.train_ds))

Train dataloader's config: 

file_path: /content/SST-2/train.tsv
batch_size: 64
shuffle: true
num_samples: -1
num_workers: 3
drop_last: false
pin_memory: false



#5 设置参数构建训练器

In [16]:
config.trainer.gpus = 1 if torch.cuda.is_available() else 0 # 通过判断cuda是否用来设置gpu的数量
config.trainer.precision = 16 # 若使用混合精度计算可以设置为16bit精度
config.trainer.amp_level = "O1"# 若使用混合精度计算这里参数需设为“01”
config.trainer.accelerator = None #使用分布式训练 in colab set "dpp" to None
config.trainer.max_epochs = 10 # 设置训练的epoch
trainer = pl.Trainer(**config.trainer) #调用pytorchlighting的Trainer将设置好参数的模型实例化

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Using native 16bit precision.


#6 设置 NeMo的日志模型管理器¶


In [17]:
print(OmegaConf.to_yaml(config.exp_manager))
exp_dir = exp_manager(trainer, config.exp_manager)
print(exp_dir)

exp_dir: null
name: TextClassification
create_tensorboard_logger: true
create_checkpoint_callback: true

[NeMo I 2021-08-22 14:12:35 exp_manager:219] Experiments will be logged at /content/nemo_experiments/TextClassification/2021-08-22_14-12-35
[NeMo I 2021-08-22 14:12:35 exp_manager:568] TensorboardLogger has been set up
/content/nemo_experiments/TextClassification/2021-08-22_14-12-35


#7 打印NeMo中可用的语言模型

In [18]:
print(nemo_nlp.modules.get_pretrained_lm_models_list())

['megatron-bert-345m-uncased', 'megatron-bert-345m-cased', 'megatron-bert-uncased', 'megatron-bert-cased', 'biomegatron-bert-345m-uncased', 'biomegatron-bert-345m-cased', 'bert-base-uncased', 'bert-large-uncased', 'bert-base-cased', 'bert-large-cased', 'bert-base-multilingual-uncased', 'bert-base-multilingual-cased', 'bert-base-chinese', 'bert-base-german-cased', 'bert-large-uncased-whole-word-masking', 'bert-large-cased-whole-word-masking', 'bert-large-uncased-whole-word-masking-finetuned-squad', 'bert-large-cased-whole-word-masking-finetuned-squad', 'bert-base-cased-finetuned-mrpc', 'bert-base-german-dbmdz-cased', 'bert-base-german-dbmdz-uncased', 'cl-tohoku/bert-base-japanese', 'cl-tohoku/bert-base-japanese-whole-word-masking', 'cl-tohoku/bert-base-japanese-char', 'cl-tohoku/bert-base-japanese-char-whole-word-masking', 'TurkuNLP/bert-base-finnish-cased-v1', 'TurkuNLP/bert-base-finnish-uncased-v1', 'wietsedv/bert-base-dutch-cased', 'distilbert-base-uncased', 'distilbert-base-uncased-

In [19]:
# HF_DATASETS_OFFLINE=1 
TRANSFORMERS_OFFLINE=1 #可以使用transformer中加载过语言模型的缓存，避免每次都重新远程加载。

# 8 模型的初始化，将中文Bert设置为预训练模型

In [20]:
config.model.language_model.pretrained_model_name = "bert-base-chinese"
model = nemo_nlp.models.TextClassificationModel(cfg=config.model, trainer=trainer)

[NeMo I 2021-08-22 14:12:56 tokenizer_utils:101] Getting HuggingFace AutoTokenizer with pretrained_model_name: bert-base-chinese, vocab_file: None, special_tokens_dict: {}, and use_fast: False


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/624 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/269k [00:00<?, ?B/s]

...

[NeMo W 2021-08-22 14:13:35 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    


[NeMo I 2021-08-22 14:13:35 text_classification_dataset:120] Read 873 examples from /content/SST-2/dev.tsv.
[NeMo I 2021-08-22 14:13:35 text_classification_dataset:238] *** Example ***
[NeMo I 2021-08-22 14:13:35 text_classification_dataset:239] example 0: ['it', "'s", 'a', 'charming', 'and', 'often', 'affecting', 'journey', '.']
[NeMo I 2021-08-22 14:13:35 text_classification_dataset:240] subtokens: [CLS] it ' s a ch ##ar ##ming and of ##ten af ##fe ##ct ##ing jo ##urn ##ey . [SEP]
[NeMo I 2021-08-22 14:13:35 text_classification_dataset:241] input_ids: 101 8233 112 161 143 9537 8458 10693 8256 8205 11598 11782 9568 8722 8221 13269 12760 8603 119 102
[NeMo I 2021-08-22 14:13:35 text_classification_dataset:242] segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
[NeMo I 2021-08-22 14:13:35 text_classification_dataset:243] input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
[NeMo I 2021-08-22 14:13:35 text_classification_dataset:244] label: 1
[NeMo I 2021-08-22 14:13:35 text_classificat

[NeMo W 2021-08-22 14:13:36 modelPT:200] You tried to register an artifact under config key=tokenizer.vocab_file but an artifact for it has already been registered.


Downloading:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertEncoder: ['cls.predictions.transform.dense.bias', 'cls.predic

#9 调用trainer中的fit函数结合数据集开始模型训练

In [21]:
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[NeMo I 2021-08-22 14:14:00 modelPT:750] Optimizer config = Adam (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.999]
        eps: 1e-08
        lr: 2e-05
        weight_decay: 0.01
    )
[NeMo I 2021-08-22 14:14:00 lr_scheduler:625] Scheduler "<nemo.core.optim.lr_scheduler.WarmupAnnealing object at 0x7f0c7b569450>" 
    will be used during training (effective maximum steps = 10530) - 
    Parameters : 
    (warmup_steps: null
    warmup_ratio: 0.1
    last_epoch: -1
    max_steps: 10530
    )



  | Name                  | Type                 | Params
---------------------------------------------------------------
0 | loss                  | CrossEntropyLoss     | 0     
1 | bert_model            | BertEncoder          | 102 M 
2 | classifier            | SequenceClassifier   | 592 K 
3 | classification_report | ClassificationReport | 0     
---------------------------------------------------------------
102 M     Trainable params
0         Non-trainable params
102 M     Total params
411.439   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

[NeMo W 2021-08-22 14:14:00 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    


...

[NeMo I 2021-08-22 15:01:55 text_classification_model:165] val_report: 
    label                                                precision    recall       f1           support   
    label_id: 0                                             86.55      69.16      76.88        428
    label_id: 1                                             75.09      89.64      81.72        444
    -------------------
    micro avg                                               79.59      79.59      79.59        872
    macro avg                                               80.82      79.40      79.30        872
    weighted avg                                            80.72      79.59      79.35        872
    


Epoch 8, global step 9476: val_loss was not in top 3


Validating: 0it [00:00, ?it/s]

[NeMo I 2021-08-22 15:07:06 text_classification_model:165] val_report: 
    label                                                precision    recall       f1           support   
    label_id: 0                                             86.71      70.09      77.52        428
    label_id: 1                                             75.67      89.64      82.06        444
    -------------------
    micro avg                                               80.05      80.05      80.05        872
    macro avg                                               81.19      79.87      79.79        872
    weighted avg                                            81.08      80.05      79.83        872
    


Epoch 9, global step 10529: val_loss was not in top 3
Saving latest checkpoint...


#10 模型的验证Evaluation

In [22]:
queries = ['"很好,很实惠"',
            '配送员很好',
            '不错，就是餐盒贵',
            '火烧夹肉好咸啊！没法吃。其他还行',
            "味道不错,配送速度比预计快",
            '没什么味道，送来的晚凉了',
            '完全不值得信任，出尔反而',
            '没发票，乱收费，订单没到就被完成了',
            '鸡蛋都坏了，凉菜也洒了',
            '很 快 ， 好 吃 ， 味 道 足 ， 量 大 ',
           '很棒！太快了',
           '和店里点的水平差很多',
           '挺快的，但是味道不太好',
           '我点的凉菜，给我送一盒热的烂白菜',
           '好服务，就是习惯百度外卖的快速',
           '挺好吃的，就是有点难吃',
           '我家的狗吃了连夜给我做了四菜一汤',
           '不错，只不过这味道对于人类来讲超前了一些，狗觉得挺好吃的']
           
     
results = model.classifytext(queries=queries, batch_size=3, max_seq_length=512)

print('The prediction results of some sample queries with the trained model:')
for query, result in zip(queries, results):
    print(f'Query : {query}')
    print(f'Predicted label: {result}')

[NeMo W 2021-08-22 15:07:51 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    


The prediction results of some sample queries with the trained model:
Query : "很好,很实惠"
Predicted label: 1
Query : 配送员很好
Predicted label: 1
Query : 不错，就是餐盒贵
Predicted label: 0
Query : 火烧夹肉好咸啊！没法吃。其他还行
Predicted label: 0
Query : 味道不错,配送速度比预计快
Predicted label: 0
Query : 没什么味道，送来的晚凉了
Predicted label: 0
Query : 完全不值得信任，出尔反而
Predicted label: 0
Query : 没发票，乱收费，订单没到就被完成了
Predicted label: 0
Query : 鸡蛋都坏了，凉菜也洒了
Predicted label: 0
Query : 很 快 ， 好 吃 ， 味 道 足 ， 量 大 
Predicted label: 0
Query : 很棒！太快了
Predicted label: 1
Query : 和店里点的水平差很多
Predicted label: 0
Query : 挺快的，但是味道不太好
Predicted label: 0
Query : 我点的凉菜，给我送一盒热的烂白菜
Predicted label: 0
Query : 好服务，就是习惯百度外卖的快速
Predicted label: 0
Query : 挺好吃的，就是有点难吃
Predicted label: 0
Query : 我家的狗吃了连夜给我做了四菜一汤
Predicted label: 0
Query : 不错，只不过这味道对于人类来讲超前了一些，狗觉得挺好吃的
Predicted label: 0


In [23]:
model.save_to("sentiment_classify_model.nemo")